In [17]:
import traceback
import sys
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [18]:
nasa_mars_news_url = "https://mars.nasa.gov/news/"
jpl_mars_space_images_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
mars_facts_url = "https://space-facts.com/mars/"
astrogeology_url = "https://astrogeology.usgs.gov"
mars_hemispheres_url = astrogeology_url + "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [20]:
class ChromeBrowser(object): 
    """
    """
    def __init__(self, url):
        self.url = url 
      
    def __enter__(self): 
        executable_path = {'executable_path': ChromeDriverManager().install()}
        self.browser = Browser('chrome', **executable_path, headless=False)
        self.browser.visit(self.url)
        return self.browser
  
    def __exit__(self, exc_type, exc_val, exc_tb):
        if (exc_type is not None):
            traceback.print_exception(exc_type, exc_val, exc_tb)
        if (self.browser is not None):
            self.browser.quit() 
        return True

In [21]:
news_title = ""
news_p = ""

with ChromeBrowser(nasa_mars_news_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    news_title = soup.find('div', class_='content_title').text
    news_p = soup.find('div', class_='article_teaser_body').text
    
print(news_title)
print()
print(news_p)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
There is no [win32] chromedriver for browser 93.0.4577 in cache
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.15/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\dowen\.wdm\drivers\chromedriver\win32\93.0.4577.15]


Mars Now

The three award recipients – Diana Trujillo, Christina Hernandez, and Clara O’Farrell – are engineers from the NASA rover team. 


In [22]:
featured_image_url = ""

with ChromeBrowser(jpl_mars_space_images_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    browser.links.find_by_partial_text("FULL IMAGE").click()
    html = browser.html
    soup = bs(html, "html.parser")
    featured_image_url = jpl_mars_space_images_url.\
    replace(
        "index.html", 
        soup.find(class_="fancybox-image")["src"]
    ) 
    
print(featured_image_url)




====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\dowen\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


In [6]:
mars_facts = pd.read_html(mars_facts_url, match="Mars")[0]
mars_facts = mars_facts.set_index("Mars - Earth Comparison", drop=True)
mars_facts = mars_facts.transpose()
mars_facts.columns = [col.replace(":","") for col in mars_facts.columns]
mars_facts = mars_facts.drop("Earth")
mars_facts = mars_facts.transpose()
mars_facts

Mars
Diameter                  6,779 km
Mass               6.39 × 10^23 kg
Moons                            2
Distance from Sun   227,943,824 km
Length of Year      687 Earth days
Temperature           -87 to -5 °C

In [23]:
mars_facts.to_json()

'{"Mars":{"Diameter":"6,779 km","Mass":"6.39 \\u00d7 10^23 kg","Moons":"2","Distance from Sun":"227,943,824 km","Length of Year":"687 Earth days","Temperature":"-87 to -5 \\u00b0C"}}'

In [25]:
hemisphere_image_urls = []

with ChromeBrowser(mars_hemispheres_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    results = soup.find(class_="result-list").find_all(class_="item")
    for result in results:
        hemisphere = {}
        hemisphere["title"] = result.find("h3").text
        browser.links.find_by_partial_text(hemisphere["title"]).click()
        html = browser.html
        soup = bs(html, "html.parser")
        hemisphere["img_url"] = astrogeology_url + soup.find(class_="wide-image")["src"]
        hemisphere_image_urls.append(hemisphere)
        browser.back()
        
for url in hemisphere_image_urls:
    print(url)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\dowen\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}
{'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}
{'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}
{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}
